## Steps:
1. Text Preprocessing and cleaning - Stemming
2. Train, Test Split
3. Text Vectorization - BOW
4. Train ML model
5. Evaluation

In [1]:
import pandas as pd

In [2]:
messages = pd.read_csv('./data/SMS Spam Detection System.csv', encoding='ISO-8859-1')
messages = messages.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1)
messages = messages.rename(columns={'v1':'label','v2': 'message' })

In [3]:
messages.head(1)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."


## Step 1: Text preprocessing and cleaning

In [4]:
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [6]:
corpus = []
for i in range(0, len(messages)):
    message = re.sub('[^a-zA-z]',' ', messages['message'][i])
    message = message.lower()
    message = message.split()
    message = [ps.stem(word) for word in message if not word in stopwords.words('english')]
    message = ' '.join(message)
    corpus.append(message)

In [7]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

## Step 2: Train - Test Split

To avoid data leakage when using a Bag-of-Words (BOW) model, it's crucial to split the data into training, validation, and testing sets before creating the BOW representation. This way, the model does not gain information from the test set during training

In [8]:
# ## Output Features

# y = pd.get_dummies(messages['label'], drop_first=True)

# y


In [9]:
messages['label_num'] = messages['label'].map({'spam': 1, 'ham': 0})
y = messages['label_num']

In [10]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: label_num, Length: 5572, dtype: int64

In [11]:
# train Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus,y, test_size=0.20)

In [12]:
y_test

3607    0
423     1
3794    0
2805    0
2563    0
       ..
3548    0
5033    0
4206    0
401     0
1736    0
Name: label_num, Length: 1115, dtype: int64

## Step 3: Text Vectorization - Create bag of words

In [13]:
## Binary Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, binary=True, ngram_range=(1,1))
X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()

In [14]:
#Binary Bag of Words with ngram 
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features=200, binary=True, ngram_range=(1,1))
# X_train = cv.fit_transform(X_train).toarray()
# X_test = cv.transform(X_test).toarray()

In [15]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float = lambda x: "%.3g" % x))


In [16]:
cv.vocabulary_

{'mode': 1255,
 'men': 1209,
 'left': 1052,
 'sea': 1781,
 'lay': 1040,
 'rock': 1725,
 'envelop': 575,
 'paper': 1443,
 'word': 2425,
 'babi': 147,
 'promis': 1577,
 'treat': 2204,
 'well': 2372,
 'bet': 184,
 'take': 2063,
 'good': 763,
 'care': 289,
 'make': 1160,
 'sure': 2040,
 'alex': 50,
 'know': 1017,
 'birthday': 194,
 'fifteen': 645,
 'minut': 1233,
 'far': 627,
 'ex': 596,
 'wife': 2395,
 'abl': 4,
 'kid': 1005,
 'want': 2339,
 'one': 1398,
 'day': 446,
 'privat': 1564,
 'account': 11,
 'statement': 1976,
 'show': 1847,
 'unredeem': 2259,
 'bonu': 220,
 'point': 1515,
 'claim': 338,
 'call': 265,
 'identifi': 892,
 'code': 353,
 'expir': 609,
 'orang': 1411,
 'custom': 428,
 'may': 1187,
 'free': 694,
 'camera': 275,
 'phone': 1483,
 'upgrad': 2267,
 'loyalti': 1129,
 'offer': 1384,
 'end': 564,
 'appli': 98,
 'opt': 1409,
 'mark': 1173,
 'forev': 679,
 'pick': 1486,
 'pain': 1437,
 'come': 362,
 'back': 148,
 'mm': 1240,
 'time': 2152,
 'dont': 518,
 'like': 1076,
 'fun': 7

## Train Multinomial Naive Bayes

In [17]:
from sklearn.naive_bayes import MultinomialNB

In [18]:
spam_detection_model = MultinomialNB().fit(X_train, y_train)

In [19]:
y_pred = spam_detection_model.predict(X_test)

## Evaluating the Model

In [20]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test, y_pred)

0.9838565022421525

In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       948
           1       0.97      0.92      0.94       167

    accuracy                           0.98      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.98      0.98      0.98      1115

